# geometry
## for manipulating geometries

In [ ]:
from gerrytools.geometry import *
from gerrytools.plotting import *
import matplotlib.pyplot as plt
import geopandas as gpd

### dual graph
This generates a graph dual to the provided geometric data (`GeoDataFrame`)

In [ ]:
%%time
vtd_shp = gpd.read_file("data/NC_vtd20/") # North Carolina VTDs
graph = dualgraph(
    vtd_shp,
)

### dissolve
This dissolves the geometric data on the column `by`. We generally use this to dissolve a set of source geometries (e.g. VTDs, blocks, etc.) to district geometries. In this case, we'll dissolve our North Carolina VTDs by county, since we don't have a district assignment column.

In [ ]:
%%time
counties = dissolve(
    vtd_shp,
    by="COUNTYFP20",
    reset_index=True, # defaults to making the result integer-indexed, not the `by` column
    keep=["TOTPOP20"], # Additional columns to keep beyond the geometry and `by` columns. Defaults to []
    aggfunc="sum", # pandas groupby function type when aggregating; defaults to "sum"
)

In [ ]:
counties

In [ ]:
from gerrytools.plotting import choropleth

counties["PERCENT_OF_MAX"] = (
    counties["TOTPOP20"] / counties["TOTPOP20"].max()
)

ax = choropleth(
    geometries=counties,
    districts=counties,
    demographic_share_col="PERCENT_OF_MAX",
    cmap="Blues",
    district_linecolor="#1F77B4",
    colorbar=False,
    figsize=(18,8),
)

In [ ]:
# show that we can now plot the county-level populations
fig, ax = plt.subplots(figsize=(18,8))
ax = counties.plot(
    ax=ax,
    column="TOTPOP20",
    cmap='Blues',
)
ax = counties.boundary.plot(ax=ax)
_ = plt.axis('off')

### unitmap and invert
`unitmap` creates a mapping from source (smaller) units to target (larger) units. `invert` inverts the provided unitmapping, mapping the target (larger) units to lists of source (smaller) units. Often we would want to do this for blocks &rarr; VTDs, but here we'll test this on VTDs &rarr; counties.

In [ ]:
%%time
mapping = unitmap((vtd_shp, "GEOID20"), (counties, "COUNTYFP20"))
inverted_mapping = invert(mapping)

In [ ]:
# VTD 37025008-00 maps to county 025
mapping['37025008-00']

In [ ]:
# county 025 maps to (touches) 40 VTDs, listed here...
inverted_mapping[25.0]

In [ ]:
mapping = unitmap((vtd_shp, "GEOID20"), (counties, "COUNTYFP20"))
inverted_mapping = invert(mapping)

print(f"mapping['37025008-00']={mapping['37025008-00']}")
print(f"inverted_mapping[25.0]={inverted_mapping[25.0]}")

### TODO:
`dispersion_updater_closure`, `minimize_parity`, `minimize_dispersion_with_parity`